In [ ]:
import json
import pandas as pd
import os
from pandas.io.json import json_normalize
import random
import pandas as pd
import math
import pickle

### JSON management

In [ ]:
def AbrirDatosMesa(fname):
    with open(fname) as json_file:  
        data = json.load(json_file)
    return data

In [ ]:
def GetDatosMesa(mesa):
    path="../../data/"
    data_name="mesa_"+'{0:06d}'.format(mesa)+'.json'
    data=AbrirDatosMesa(path+data_name)
    return data

In [ ]:
def ExtractDataActa(acta,data):
    partidosC1=[] #nombres de partidos en la primer columna
    votosC1=[] #votos partidos en la primer columna
    partidosC2=[] #nombres de partidos en la segunda columna
    votosC2=[] #votos partidos en la segunda columna
    #print(acta)
    
    b=None
    for dacta in data['TE']:
        if 'VOTOSTE'+str(acta-1) in dacta:
            b=dacta['VOTOSTE'+str(acta-1)]
            break
    
    if not b:
        return partidosC1,votosC1,partidosC2,votosC2    
    #print(acta)
    maxA=24-1
    for i,val in enumerate(b):
        if i>maxA:
            val['D']=''.join([i for i in val['D'] if i.isalpha()])
            partidosC2.append(val['D'])
            votosC2.append(int(val['V']))
        else:
            val['D']=''.join([i for i in val['D'] if i.isalpha()])
            partidosC1.append(val['D'])
            votosC1.append(int(val['V']))
    return partidosC1,votosC1,partidosC2,votosC2

In [ ]:
def ProcessFile(mesa):
    print("Mesa",mesa)
    path="../../data/"
    out_path='./'
    mfname="mesa_"+'{0:06d}'.format(mesa)+'.json'
    
    exists = os.path.isfile(path+mfname)
    if not exists:
        print("Datos ",mfname," NO EXISTE")
        return (True,-1)
    
    mdata=GetDatosMesa(mesa)
    total_actas={}
    for acta in range(1,6):       
        partidosC1,votosC1,partidosC2,votosC2=ExtractDataActa(acta,mdata)
        total_actas[acta]=((partidosC1,votosC1,partidosC2,votosC2))
    return total_actas

### Creates or Loads a "database" made of nested dict and lists out of JSONs

In [ ]:
def CreateLoadDatabase():
    fname='database.p'
    path="./"
    
    exists = os.path.isfile(path+fname)
    
    if exists:
        with open(path+fname, 'rb') as handle:
            todas_mesas=pickle.load(handle)
    else:        
        todas_mesas={}
        tot_mesas=21100
        for i in range(1,tot_mesas):
            mesa=ProcessFile(i)
            todas_mesas[i]=mesa
        with open(path+fname, 'wb') as handle:
            pickle.dump(todas_mesas,handle)
    return todas_mesas

In [ ]:
database=CreateLoadDatabase()

## Mturk data Management

In [ ]:
fname='Batch_3693194_batch_results.csv'
df=pd.read_csv(fname, sep=',')
#df=df[0:9] #small test
df=df[['Input.imgName','Answer.IsDifficult.difficult','Answer.tagA01','Answer.tagA02','Answer.tagA03','Answer.tagA04','Answer.tagA05','Answer.tagA06','Answer.tagA07','Answer.tagA08','Answer.tagA09','Answer.tagA10','Answer.tagA11','Answer.tagA12','Answer.tagA13','Answer.tagA14','Answer.tagA15','Answer.tagA16','Answer.tagA17','Answer.tagA18','Answer.tagA19','Answer.tagA20','Answer.tagA21','Answer.tagA22','Answer.tagA23','Answer.tagA24']]
df.columns = ['name','difficult','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24']

#### Calculate consensus: -1 without consensus, -2 error

In [ ]:
def aggFunc(series):
    v=series.value_counts(dropna=False)
    try:
        if v.iloc[0]<2:
            return -1
        else:
            return v.index[0]
    except:
        return -2

In [ ]:
b=df.groupby('name').agg(aggFunc).reset_index().copy(deep=True)

#### Extract measa and acta from jpeg filename

In [ ]:
b['mesa']=b['name'].str[:5]
b['acta']=b['name'].str[-5]

#### Keep relevant results and convert to integers

In [ ]:
b=b[['mesa','acta','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24']]
b=b.astype(int)

#### Count fails and lack of consesus

In [ ]:
fail_count=0
sconsenso_count=0
site='https://elecciones2019.s3.us-east-2.amazonaws.com/data_mturk/'
for _, data in b.iterrows():
    acta_num=data['acta']
    mesa_num=data['mesa']    
    votos_tse=database[mesa_num][acta_num][1]
    mtvotes=list(data[2:len(votos_tse)+2])
    if -1 in mtvotes:
        sconsenso_count+=1
        print('##Mesa {} en acta {} sin consenso'.format(mesa_num,acta_num))
    elif votos_tse!=mtvotes:
        fname='{0:06d}'.format(mesa_num*10+acta_num)+'.jpg'
        fail_count+=1
        print('Mesa {} en acta {} no cuadra'.format(mesa_num,acta_num))
        print(site+fname)
        print('TSE ',votos_tse)
        print('MTURK ',mtvotes)
        print('-'*10)

In [ ]:
print("Fail ",fail_count)
print("CFail ",sconsenso_count)

In [ ]:
print("Mesas mturk",len(b))

#### Check indivdiaul mesa

In [ ]:
mesa=1
acta=1
fname='{0:06d}'.format(mesa*10+acta)+'.jpg'
site='https://elecciones2019.s3.us-east-2.amazonaws.com/data_mturk/'
print(site+fname)
database[mesa][acta][1]